In [1]:
from data_processing import *

In [2]:
translated_filtered, not_translated_filtered = merge_pages()

(153786, 4)
(84014, 4)


/Users/markokostiv/ucu/mining_datasets/wiki/data_processing.py:16: RuntimeWarning: Mean of empty slice
  average = data.drop(columns=["page_id", "title"]).apply(lambda x: np.nanmean(x), axis=1)


(153787, 10)
(84010, 10)


In [3]:
translated_filtered.head()

,wikidata,title,page_id,len,rev_count,image_links_count,translations_count,days_old,contributions,average_views
0,371,!!!,425480,3953,28,2.0,25.0,3399,21,43.333333
1,1622767,!T.O.O.H.!,425157,3031,14,2.0,7.0,3400,13,23.000000
2,1075363,'39,2014472,8083,12,4.0,14.0,1044,4,51.333333
3,4544561,(A)live,2211095,2207,10,5.0,3.0,746,7,19.833333
4,3153272,(ISC)²,2699656,10572,14,1.0,5.0,95,2,68.000000


In [4]:
not_translated_filtered.head()

,wikidata,title,page_id,len,rev_count,image_links_count,translations_count,days_old,contributions,average_views
0,4027682,!ФЕСТ,432740,21769,200,10.0,1.0,3383,68,501.833333
1,12061550,!Чидро,590602,2365,12,4.0,NaN,3145,10,22.666667
2,30196568,$uicideboy$,2610007,4878,9,2.0,6.0,207,6,421.500000
3,11511,&RQ,156371,4659,22,2.0,2.0,4110,16,46.166667
4,25447349,'Cichlasoma',357578,6104,55,3.0,NaN,3598,17,16.500000


In [5]:
translated_data = translated_filtered.drop(columns = ["wikidata", "page_id"])
not_translated_data = not_translated_filtered.drop(columns = ["wikidata", "page_id"])

In [6]:
translated_data.corr()

,len,rev_count,image_links_count,translations_count,days_old,contributions,average_views
len,1.000000,0.529863,0.347939,0.161069,-0.027650,0.443487,0.386506
rev_count,0.529863,1.000000,0.459824,0.560104,0.453438,0.911880,0.622187
image_links_count,0.347939,0.459824,1.000000,0.389164,0.092488,0.419105,0.301966
translations_count,0.161069,0.560104,0.389164,1.000000,0.536578,0.654192,0.371287
days_old,-0.027650,0.453438,0.092488,0.536578,1.000000,0.590117,0.213973
contributions,0.443487,0.911880,0.419105,0.654192,0.590117,1.000000,0.652450
average_views,0.386506,0.622187,0.301966,0.371287,0.213973,0.652450,1.000000


In [7]:
not_translated_data.corr()

,len,rev_count,image_links_count,translations_count,days_old,contributions,average_views
len,1.000000,0.313578,0.126453,0.099935,-0.182553,0.205947,0.112181
rev_count,0.313578,1.000000,0.284095,0.166706,0.396169,0.847570,0.391178
image_links_count,0.126453,0.284095,1.000000,0.097565,0.116016,0.295442,0.057055
translations_count,0.099935,0.166706,0.097565,1.000000,0.033661,0.227914,0.231244
days_old,-0.182553,0.396169,0.116016,0.033661,1.000000,0.592222,0.121540
contributions,0.205947,0.847570,0.295442,0.227914,0.592222,1.000000,0.414797
average_views,0.112181,0.391178,0.057055,0.231244,0.121540,0.414797,1.000000


In [8]:
def normalize(data):
    columns = ["rev_count", "image_links_count", "days_old", "contributions", "average_views", "translations_count"]
    for column in columns:
        data[column] = data[column]/data[column].max()

normalize(not_translated_data)
normalize(translated_data)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
translated_data['class'] = 1
not_translated_data['class'] = 0

data = pd.concat([translated_data, not_translated_data])
data.fillna(0, inplace=True)
data.head()

,title,len,rev_count,image_links_count,translations_count,days_old,contributions,average_views,class
0,!!!,3953,0.009428,0.005682,0.086505,0.642898,0.030129,0.000668,1
1,!T.O.O.H.!,3031,0.004714,0.005682,0.024221,0.643087,0.018651,0.000355,1
2,'39,8083,0.004040,0.011364,0.048443,0.197465,0.005739,0.000792,1
3,(A)live,2207,0.003367,0.014205,0.010381,0.141101,0.010043,0.000306,1
4,(ISC)²,10572,0.004714,0.002841,0.017301,0.017969,0.002869,0.001049,1


In [11]:
columns_to_drop = ['class', 'title', 'rev_count']

labels = data['class']
dataset = data.drop(columns=columns_to_drop, axis = 1)
dataset.head()

,len,image_links_count,translations_count,days_old,contributions,average_views
0,3953,0.005682,0.086505,0.642898,0.030129,0.000668
1,3031,0.005682,0.024221,0.643087,0.018651,0.000355
2,8083,0.011364,0.048443,0.197465,0.005739,0.000792
3,2207,0.014205,0.010381,0.141101,0.010043,0.000306
4,10572,0.002841,0.017301,0.017969,0.002869,0.001049


In [12]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels.values, test_size=0.3, random_state=42)

In [13]:
from xgboost import XGBClassifier

In [14]:
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
from sklearn.svm import SVC

In [16]:
# svm_classifier = SVC()
# svm_train = [-1 if y == 0 else 1 for y in y_train]
# svm_classifier.fit(X_train, svm_train)

In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def predict(model, x_test):
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    return predictions

def evaluate_results(y_test, predictions):
    accuracy = accuracy_score(y_test, predictions)
    matrix = confusion_matrix(y_test, predictions)
    return accuracy, matrix

In [18]:
# 94.17 with contributions
# 94.12 with rev_count
predictions = predict(xgb_classifier, X_test)
accuracy, matrix = evaluate_results(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(matrix)

Accuracy: 99.18%
[[25078   360]
 [  227 45675]]


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
# 74.74 with contributions
# 74.40 with rev_count
# predictions = predict(svm_classifier, X_test)
# svm_test = [-1 if y == 0 else 1 for y in y_test]
# accuracy, matrix = evaluate_results(svm_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))
# print(matrix)

In [20]:
from sklearn.dummy import DummyClassifier
dummy_classifier = DummyClassifier()
dummy_classifier.fit(X_train, y_train)

predictions = predict(dummy_classifier, X_test)
accuracy, matrix = evaluate_results(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(matrix)

Accuracy: 54.23%
[[ 8940 16498]
 [16157 29745]]


In [21]:
# svm_linear_classifier = SVC(kernel = "linear")
# svm_linear_classifier.fit(X_train, svm_train)

In [22]:
# predictions = predict(svm_linear_classifier, X_test)
# svm_test = [-1 if y == 0 else 1 for y in y_test]
# accuracy, matrix = evaluate_results(svm_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))
# print(matrix)

In [23]:
from sklearn.cluster import KMeans

In [24]:
cluster_data = translated_data.drop(columns=columns_to_drop)
cluster_data.fillna(0, inplace=True)

print(cluster_data.head())
k_means = KMeans(n_clusters=1, random_state=0).fit(cluster_data)

     len  image_links_count  translations_count  days_old  contributions  \
0   3953           0.005682            0.086505  0.642898       0.030129   
1   3031           0.005682            0.024221  0.643087       0.018651   
2   8083           0.011364            0.048443  0.197465       0.005739   
3   2207           0.014205            0.010381  0.141101       0.010043   
4  10572           0.002841            0.017301  0.017969       0.002869   

   average_views  
0       0.000668  
1       0.000355  
2       0.000792  
3       0.000306  
4       0.001049  


In [25]:
k_means.cluster_centers_

array([[7.16486427e+03, 1.55969857e-02, 8.77394054e-02, 4.43085109e-01,
        1.78502773e-02, 1.63605058e-03]])

In [26]:
ranking_data = not_translated_data
ranking_data.fillna(0, inplace=True)
ranking_data.head()

,title,len,rev_count,image_links_count,translations_count,days_old,contributions,average_views,class
0,!ФЕСТ,21769,0.280505,0.057471,0.009615,0.640356,0.348718,0.034385,0
1,!Чидро,2365,0.016830,0.022989,0.000000,0.595306,0.051282,0.001553,0
2,$uicideboy$,4878,0.012623,0.011494,0.057692,0.039182,0.030769,0.028880,0
3,&RQ,4659,0.030856,0.011494,0.019231,0.777967,0.082051,0.003163,0
4,'Cichlasoma',6104,0.077139,0.017241,0.000000,0.681052,0.087179,0.001131,0


In [48]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

the_best_translated = k_means.cluster_centers_.reshape(1,-1)

def similarity(features):
    the_best_translated = k_means.cluster_centers_.reshape(1,-1)
    features = features.drop(labels=columns_to_drop, axis=0).values
    features = features.reshape(1,-1)
    similarity = cosine_similarity(features, the_best_translated)
    return similarity

# sim = similarity(np.array([0, 0, 0, 0, 0]))
ranking_with_similarity = ranking_data.copy()
ranking_with_similarity['xgb_ranking'] = xgb_classifier.predict_proba(ranking_data.drop(labels=columns_to_drop, axis=1))[:,[1]]
ranking_with_similarity['cosine_ranking'] = ranking_data.apply(lambda row: similarity(row), axis=1)

In [49]:
ranking_with_similarity.sort_values('cosine_ranking', ascending = False).head()

,title,len,rev_count,image_links_count,translations_count,days_old,contributions,average_views,class,xgb_ranking,cosine_ranking
71277,Стурдза,2604,0.004208,0.005747,0.028846,0.163733,0.010256,0.000879,0,0.061763,[[0.9999999999977275]]
81707,Шибляк,1552,0.002805,0.005747,0.019231,0.094643,0.005128,0.002147,0,0.011741,[[0.9999999999974577]]
2407,ED74,3743,0.002805,0.011494,0.038462,0.234904,0.010256,0.001450,0,0.167390,[[0.9999999999971956]]
67905,Селти,7073,0.021038,0.028736,0.076923,0.433277,0.020513,0.000788,0,0.479392,[[0.9999999999970225]]
56746,Носилове,5661,0.005610,0.022989,0.057692,0.345826,0.015385,0.000331,0,0.101732,[[0.9999999999957917]]


In [50]:
final_results = ranking_with_similarity.sort_values('xgb_ranking', ascending = False)

In [52]:
final_results = final_results.drop(columns="cosine_ranking")
final_results.to_csv("ranking_final_results.csv", index = False)

In [53]:
final_results.head()

,title,len,rev_count,image_links_count,translations_count,days_old,contributions,average_views,class,xgb_ranking
58901,П'ятирічка,8020,0.032258,0.000000,0.201923,0.741435,0.030769,0.056779,0,0.981484
32835,Гребінцеві,7266,0.039271,0.045977,0.221154,0.831535,0.046154,0.007194,0,0.977610
32364,Горизонталь,3314,0.023843,0.017241,0.317308,0.791028,0.030769,0.024472,0,0.975810
34100,Даманові,11985,0.009818,0.057471,0.153846,0.494416,0.015385,0.002535,0,0.975549
76471,Ушача,11219,0.005610,0.017241,0.096154,0.312512,0.010256,0.000605,0,0.971391
